# Implementation of several metrics that measure distribution divergence

**Important!** We measure divergence of multiple distributions & the distributions are multi-variate!!

* Fit a **Logistic/Linear Regression** model and check how well we could fit it.
* **Separability index**: for each point, take its nearest neighbors and check if the classes match
* **Class scatter matrices/measure**
* **Direct class separability measure, DCSM**
* **Bhattacharya distance **
* **Bin the distributions and measure Collective entropy**
* **KL & JS Divergence**
* **Earth-Movers-Distance (EMD)** of histograms

In [96]:
X = [[8.94427191,8.94427191,8.94427191],
 [8.94427191,8.94427191,8.94427191],
 [8.94427191,8.94427191,8.94427191],
 [8.94427191,8.94427191,8.94427191],
 [8.94427191,8.94427191,8.94427191],
 [8.94427191,8.94427191,8.94427191],
 [8.94427191,8.94427191,8.94427191],
 [8.94427191,8.94427191,8.94427191],
 [8.94427191,8.94427191,8.94427191],
 [8.94427191,8.94427191,8.94427191],
 [8.94427191,8.94427191,8.94427191],
 [8.94427191,8.94427191,8.94427191],
 [8.94427191,8.94427191,8.94427191],
 [8.94427191,8.94427191,8.94427191],
 [8.94427191,8.94427191,8.94427191],
 [8.94427191,8.94427191,8.94427191],
 [8.94427191,8.94427191,8.94427191],
 [8.94427191,8.94427191,8.94427191],
 [8.94427191,8.94427191,8.94427191],
 [8.94427191,8.94427191,8.94427191],
 [8.94427191,8.94427191,8.94427191],
 [8.94427191,8.94427191,8.94427191],
 [8.94427191,8.94427191,8.94427191],
 [8.94427191,8.94427191,8.94427191],
 [8.94427191,8.94427191,8.94427191],
 [8.94427191,8.94427191,8.94427191],
 [8.94427191,8.94427191,8.94427191]]
y = [1,0,0,0,1,1,1,1,0,1,1,1,0,0,0,0,1,1,1,1,1,1,0,0,0,1,1]


In [101]:
import numpy as np
from collections import Counter, defaultdict

def bhattacharyya(X, y, cells_per_dim=10):
    # Calculate lower and upper bound for each dimension
    bounds = {}
    widths = {}
    col_to_del = []
    cntr = 0
    for d in range(X.shape[1]):
        mi, ma = min(X[:, d]), max(X[:, d])
        if mi == ma:
            col_to_del.append(d)
        else:
            bounds[cntr] = (mi, ma)
            widths[cntr] = (ma - mi) / cells_per_dim
            cntr += 1
    
    X = np.delete(X, col_to_del, axis=1)

    if X.shape[1] == 0:
        return 1
            
    # For each datapoint, calculate its cell in the hypercube
    cell_assignment_counts = []
    label_to_idx = {}
    for i, l in enumerate(set(y)): 
        cell_assignment_counts.append(defaultdict(int))
        label_to_idx[l] = i
        
    unique_assignments = set()
    for point_idx, l in zip(range(X.shape[0]), y):
        assignment = []
        for dim_idx in range(X.shape[1]):
            val = X[point_idx, dim_idx]
            cell = (val - bounds[dim_idx][0])//widths[dim_idx]
            assignment.append(cell)
        cell_assignment_counts[label_to_idx[l]][tuple(assignment)] += 1
        unique_assignments.add(tuple(assignment))
        
    totals = {}
    for l in set(y):
        totals[l] = sum(cell_assignment_counts[label_to_idx[l]].values())
    
    print(cell_assignment_counts)
    
    dist = 0
    for assign in unique_assignments:
        temp = 1
        for l in set(y):
            temp *= cell_assignment_counts[label_to_idx[l]][assign] / totals[l]
        dist += (temp * (temp != 1)) ** (1 / len(totals))
    
    return dist
            
    
from scipy.spatial.distance import pdist, euclidean
def davies_bouldin(X, labels):
    n_cluster = len(np.bincount(labels))
    cluster_k = [X[labels == k] for k in range(n_cluster)]
    centroids = [np.mean(k, axis = 0) for k in cluster_k]
    variances = [np.mean([euclidean(p, centroids[i]) for p in k]) for i, k in enumerate(cluster_k)]
    db = []

    for i in range(n_cluster):
        for j in range(n_cluster):
            if j != i:
                d = euclidean(centroids[i], centroids[j])
                if d == 0:
                    db.append(0)
                else:
                    db.append((variances[i] + variances[j]) / euclidean(centroids[i], centroids[j]))
                    
    return(np.max(db) / n_cluster)
    
#bhattacharyya(np.array(X), np.array(y))
davies_bouldin(np.array(X), np.array(y))

[array([8.94427191, 8.94427191, 8.94427191]), array([8.94427191, 8.94427191, 8.94427191])]
--> [0, 0]


0.0

In [70]:
from collections import defaultdict
import numpy as np

def partition_feature_vectors(X, y):
    distributions = defaultdict(list)
    for feature_vector, label in zip(X, y):
        distributions[label].append(feature_vector)
    return distributions.values()

from sklearn.linear_model import LogisticRegression
def linear_regression(X, y):
    lr = LogisticRegression()
    lr.fit(X, y)
    return lr.score(X, y)

from sklearn.neighbors import KDTree, BallTree
def separability_index(X, y):
    kdt = BallTree(X, metric='euclidean')
    nearest_neighbors = kdt.query(X, k=2, return_distance=False)[:, 1]
    matches = 0
    for i in range(len(X)):
        matches += (y[i] == y[nearest_neighbors[i]])
    return matches / len(X)

def class_scatter_matrix(X, y):
    # SOURCE 1: https://datascience.stackexchange.com/questions/11554/varying-results-when-calculating-scatter-matrices-for-lda
    # SOURCE 2: https://open.uct.ac.za/bitstream/item/17644/Mthembu_Conference_2004.pdf?sequence=1
    
    # A 10000x2000 dataframe takes LR 35s, this method takes 2s
    # --> 2000 columns (which is the bottleneck) is definitely a LARGE upper bound for the shapelet extraction usecase
    
    # 2500x2000 dataframe takes LR 5.5s and this method 1s
    
    # Construct a mean vector per class
    mean_vecs = {}
    for label in set(y):
        mean_vecs[label] = np.mean(X[y==label], axis=0)
        
    # Construct the within class matrix (S_w)
    d = X.shape[1]
    S_w = np.zeros((d, d))
    for label, mv in zip(set(y), mean_vecs):
        class_scatter = np.cov(X[y==label].T)
        S_w += class_scatter
        
    # Construct an overall mean vector
    mean_overall = np.mean(X, axis=0)
    
    # Construct the between class matrix (S_b)
    S_b = np.zeros((d, d))
    for i in mean_vecs:
        mean_vec = mean_vecs[i]
        n = X[y==i, :].shape[0]
        mean_vec = mean_vec.reshape(d, 1)
        mean_overall = mean_overall.reshape(d, 1)
        S_b += n * (mean_vec - mean_overall).dot((mean_vec - mean_overall).T)
        
    # Differs a bit from proposed formule (where we divide by trace(S_w)), we make sure it is bounded by 1.0!
    return np.trace(S_b) / np.trace(S_w + S_b)

def bhattacharya_distance(X, y):
    # TODO: Rewrite this code --> Iterate over each datapoint and calculate the according cell to solve curse of dim
    # http://www.eecs.yorku.ca/research/techreports/2015/EECS-2015-02.pdf
    histograms = {}
    for c in set(y):
        # This crashes!!!!
        histograms[c] = np.histogramdd(X[:, :20], normed=True)[0].flatten()
        
    d = 0
    for i in range(len(histograms[y[0]])):
        s = 1
        for c in histograms:
            s *= histograms[c][i]
        d += s ** (1/len(histograms))
            
    return d

In [71]:
from sklearn.datasets import make_classification
import time

In [73]:
class_seps = [0.05, 0.25, 0.5, 1.0]
samples = [100, 1000, 2500, 10000]
features = [20, 100, 500, 2000]
for nr_samples in samples:
    for nr_features in features:
        lr_scores = []
        csm_scores = []
        for sep in class_seps:
            X, y = make_classification(n_samples=nr_samples, n_features=nr_features, class_sep=sep, n_classes=3, n_informative=5)
            print('#Samples = {} || #Features = {} || Separation = {}'.format(nr_samples, nr_features, sep))
            start = time.time()
            score = linear_regression(X, y)
            end = time.time()
            lr_scores.append(score)
            print('LR score = {} || time = {}'.format(score, end-start))
            start = time.time()
            score = class_scatter_matrix(X, y)
            if score > 1.0:
                print('Not bounded...')
            end = time.time()
            csm_scores.append(score)
            print('SI score = {} || time = {}'.format(score, end-start))
        #print(np.corrcoef(lr_scores, class_seps), np.corrcoef(csm_scores, class_seps)[0][1])

#Samples = 100 || #Features = 20 || Separation = 0.05
LR score = 0.54 || time = 0.003136873245239258
SI score = 0.4925367823724272 || time = 0.0015692710876464844
#Samples = 100 || #Features = 20 || Separation = 0.25
LR score = 0.55 || time = 0.00263214111328125
SI score = 0.4426408198846103 || time = 0.0010442733764648438
#Samples = 100 || #Features = 20 || Separation = 0.5
LR score = 0.59 || time = 0.0024421215057373047
SI score = 0.414602055494247 || time = 0.0010521411895751953
#Samples = 100 || #Features = 20 || Separation = 1.0
LR score = 0.76 || time = 0.0024251937866210938
SI score = 0.7690458416694668 || time = 0.0016279220581054688
#Samples = 100 || #Features = 100 || Separation = 0.05
LR score = 1.0 || time = 0.012335062026977539
SI score = 0.4065327635422319 || time = 0.002017498016357422
#Samples = 100 || #Features = 100 || Separation = 0.25
LR score = 1.0 || time = 0.010277509689331055
SI score = 0.4091861251720159 || time = 0.009413003921508789
#Samples = 100 || #Feature

LR score = 0.613 || time = 0.17855405807495117
SI score = 0.9937306187582808 || time = 0.006773233413696289
#Samples = 10000 || #Features = 100 || Separation = 0.05
LR score = 0.3841 || time = 0.3342397212982178
SI score = 0.47580986967018424 || time = 0.023267507553100586
#Samples = 10000 || #Features = 100 || Separation = 0.25
LR score = 0.4631 || time = 0.32274842262268066
SI score = 0.829036975864065 || time = 0.02506875991821289
#Samples = 10000 || #Features = 100 || Separation = 0.5
LR score = 0.5889 || time = 0.39772629737854004
SI score = 0.9449091964766645 || time = 0.025813579559326172
#Samples = 10000 || #Features = 100 || Separation = 1.0
LR score = 0.6162 || time = 0.49792027473449707
SI score = 0.9791902859441074 || time = 0.02372288703918457
#Samples = 10000 || #Features = 500 || Separation = 0.05
LR score = 0.4577 || time = 1.3018431663513184
SI score = 0.420656572106113 || time = 0.2711014747619629
#Samples = 10000 || #Features = 500 || Separation = 0.25
LR score = 0.4